<a href="https://colab.research.google.com/github/zizekw/Hexapawn_assignment/blob/master/Hexapawn_Minimax_BZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SCS 3547
### William Zizek

Original code adapted from or included in full with credit to https://github.com/maxpumperla/deep_learning_and_the_game_of_go and the book "Deep Learning and the Game of Go". Consultation with the discussion board and classmates was further used for assistance. 

The purpose of this notebook is to adapt the `dlgo` code to play the game of Hexapawn while using a minimax algorithm for the bot. The objective of the minimax algorithm is to choose a move that maximizes advantage while minimizing the opponent's ability to cause damage based on the confines of the game.

---

#### To run the game, execute all code in this notebook and play at the bottom of the page!

---

#### To win the game, you must either reach the other end of the board OR you must block your opponent from moving OR "take-over" all of your opponents pieces.


In [0]:
!pip install dlgo

Player Definitions

In [0]:
import enum
from collections import namedtuple

__all__ = [
    'Player',
    'Point',
]


class Player(enum.Enum):
    x = 'W'
    o = 'B'

    @property
    def other(self):
        return Player.x if self == Player.o else Player.o

    @property
    def prefix(self):
        return 'W' if self == Player.x else 'B'

    @staticmethod
    def from_prefix(prefix):
        return Player.x if prefix == 'x' else Player.o

Board Construction

In [0]:
import copy

__all__ = [
    'Board',
    'GameState',
    'Move',
]


class IllegalMoveError(Exception):
    pass


BOARD_SIZE = 3
ROWS = tuple(range(1, BOARD_SIZE + 1))
COLS = tuple(range(1, BOARD_SIZE + 1))
PLAYERS = (Player.x, Player.o)

class Board:
    def __init__(self):
        # define the board
        self.m_map = {}
        self.m_map.update({'{}{}'.format(Player.x.prefix, idx): Point(1, idx) for idx in COLS})
        self.m_map.update({'{}{}'.format(Player.o.prefix, idx): Point(BOARD_SIZE, idx) for idx in COLS})
        self.next_player = Player.x
        self.possibles = {}
        self.win = {}
        self.update()

    def place(self, player, move):
        assert move in self.possibles[player]

        to_pop = self.grid.get(move.point)
        if to_pop:
            self.m_map.pop(to_pop)
        self.m_map[move.m] = move.point
        self.next_player = player.other
        self.update()

    @staticmethod
    def is_on_grid(point):
        return 1 <= point.row <= BOARD_SIZE and \
            1 <= point.col <= BOARD_SIZE

    def get(self, point):
        return self.grid.get(point)

    def _render_grid(self):
        self.grid = {val: key for key, val in self.m_map.items()}

    def update(self):
        def _mins(player):
            return ('{}{}'.format(player.prefix, idx) for idx in COLS)

        def _row_diff(player):
            return 1 if player == Player.x else -1

        def _row_reach(player):
            return 1 if player == Player.o else BOARD_SIZE

        def _move_point(point, row_diff, col_diff):
            return Point(point[0] + row_diff, point[1] + col_diff)

        def _check_capture(player, m, col_diff):
            row_diff = _row_diff(player)
            new_point = _move_point(self.m_map[m], row_diff, col_diff)
            occupant = self.grid.get(new_point)
            if occupant and Player.from_prefix(occupant[:1]) != player:
                return Move(m, new_point)
            else:
                return None

        def _check_forward(player, m):
            row_diff = _row_diff(player)
            new_point = _move_point(self.m_map[m], row_diff, 0)
            if not self.grid.get(new_point):
                return Move(m, new_point)
            else:
                return None

        def _m_possibles(player, m):
            return list(filter(bool, (
                [_check_capture(player, m, col_diff) for col_diff in [-1, 1]] +
                [_check_forward(player, m)])))

        def _m_reach(player, m):
            return self.m_map[m][0] == _row_reach(player)

        def _analyze_player(player):
            alive_ms = list(filter(lambda m: m in self.m_map, _mins(player)))
            possibles = sum([_m_possibles(player, m) for m in alive_ms], [])
            reach = any([_m_reach(player, m) for m in alive_ms])

            stuck = (not possibles) and player == self.next_player
            die = (not alive_ms) or stuck

            return possibles, reach, die

        self._render_grid()

        possibles = {}
        win = {player: False for player in PLAYERS}
        for player in PLAYERS:
            possibles[player], reach, die = _analyze_player(player)
            win[player] |= reach
            win[player.other] |= die

        self.possibles.update(possibles)
        self.win.update(win)


class Move:
    def __init__(self, m, point):
        self.m = m
        self.point = point

    def __eq__(self, other):
        if isinstance(other, Move):
            return self.m == other.m and self.point == other.point
        return False

    def __repr__(self):
        return 'Move {} to {}'.format(self.m, self.point)


class GameState:
    def __init__(self, board, next_player, move):
        self.board = board
        self.next_player = next_player
        self.last_move = move

    def apply_move(self, move):
        """Return the new GameState after applying the move."""
        next_board = copy.deepcopy(self.board)
        next_board.place(self.next_player, move)
        return GameState(next_board, self.next_player.other, move)

    @classmethod
    def new_game(cls):
        board = Board()
        return GameState(board, Player.x, None)

    def legal_moves(self):
        return self.board.possibles[self.next_player]

    def is_over(self):
        return any(self.board.win.values())

    def winner(self):
        return next(p for p in PLAYERS if self.board.win[p])

# Let's play...

In [0]:
# lets play

class Point(namedtuple('Point', 'row col')):
    def __deepcopy__(self, memodict={}):
        # These are very immutable.
        return self
        
from dlgo import minimax

def print_board(board):
    print('     Col1 Col2 Col3')
    for row in (1, 2, 3):
        pieces = []
        for col in (1, 2, 3):
            m = board.get(Point(row, col))
            pieces.append(m or '  ')
        print('Row%d  %s' % (row, ' | '.join(pieces)))
    print()


def main():
    game = GameState.new_game()

    human_player = Player.x 
    # bot_player = Player.o 

    bot = minimax.MinimaxAgent()

    while not game.is_over():
        if game.next_player == human_player:
            print_board(game.board)
            print("Your turn!")
            moves = game.legal_moves()
            print('\n'.join('{} {}'.format(idx, m) for idx, m in enumerate(moves)))
            human_select = int(input('-- ').strip())
            move = moves[human_select]
        else:
            print_board(game.board)
            print("bot time")
            move = bot.select_move(game)
        game = game.apply_move(move)

    print_board(game.board)
    winner = game.winner()
    if winner is None:
        print("It's a draw.")
    else:
        p_win = str(winner)
        if p_win == "Player.x": 
          tmp = "u"
        else:
          tmp = "bot" 
        print('Winner: ' + tmp)


if __name__ == '__main__':
    main()

Using TensorFlow backend.


     Col1 Col2 Col3
Row1  W1 | W2 | W3
Row2     |    |   
Row3  B1 | B2 | B3

Your turn!
0 Move W1 to Point(row=2, col=1)
1 Move W2 to Point(row=2, col=2)
2 Move W3 to Point(row=2, col=3)
-- 2
     Col1 Col2 Col3
Row1  W1 | W2 |   
Row2     |    | W3
Row3  B1 | B2 | B3

bot time
     Col1 Col2 Col3
Row1  W1 | W2 |   
Row2     | B2 | W3
Row3  B1 |    | B3

Your turn!
0 Move W1 to Point(row=2, col=2)
1 Move W1 to Point(row=2, col=1)
2 Move W2 to Point(row=2, col=3)
-- 1
     Col1 Col2 Col3
Row1     | W2 |   
Row2  W1 | B2 | W3
Row3  B1 |    | B3

Winner: u




---

